Classificar os commits

In [ ]:
import pandas as pd
import re

file_path = "/content/sample_data/1_java_commits.csv"
df = pd.read_csv(file_path)

def classify_code(language, comment, source_code):
    language = str(language) if pd.notnull(language) else ""
    comment = str(comment) if pd.notnull(comment) else ""
    source_code = str(source_code) if pd.notnull(source_code) else ""

    if language.lower() != "java":
        return "N/A"
    if re.search(r"(System\.out\.print|debugger|hardcoded)", source_code, re.IGNORECASE):
        return "Código com Débito Técnico"
    elif re.search(r"(exec|runtime\.exec|statement\(\))", source_code, re.IGNORECASE):
        return "Vulnerabilidade (Injeção de Comandos - OWASP)"
    elif "password" in source_code.lower() and "encrypt" not in source_code.lower():
        return "Vulnerabilidade (Exposição de Dados Sensíveis - OWASP)"
    elif "try" in source_code.lower() and "catch" not in source_code.lower():
        return "Bug Crítico"
    elif "TODO" in comment.upper() or "FIXME" in comment.upper():
        return "Bug Normal"
    else:
        return "Código em Conformidade"

df["Issue Type"] = df.apply(lambda row: classify_code(row["Language"], row["Comment"], row["SourceCode"]), axis=1)
output_path = "/content/classified_code.csv"
df.to_csv(output_path, index=False)
print(f"Arquivo classificado salvo em: {output_path}")


Arquivo classificado salvo em: /content/classified_code.csv


Parametrizações de acesso e arquivo

In [ ]:
# !pip install openai pandas
# !pip install --upgrade openai



import pandas as pd

# Carregar o arquivo classificado
file_path = "/content/sample_data/classified_code.csv"
df = pd.read_csv(file_path)



LLM que irá gerar correções

In [ ]:
import openai

openai.api_key = "YOUR KEY"
def generate_fix(issue_type, source_code, comment):
    prompt = f"""
    O código abaixo foi identificado como '{issue_type}'. Por favor, forneça um código corrigido e explique a correção:

    Comentário: {comment}
    Código original: {source_code}

    Código corrigido:
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Você é um especialista em correção de códigos."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1500,
            temperature=0.7
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Erro ao gerar correção: {str(e)}"

issue_type = "Bug Crítico"
source_code = "public int divide(int a, int b) { return a / b; }"
comment = "Este código não lida com divisão por zero."
fix = generate_fix(issue_type, source_code, comment)

# Imprimir a resposta gerada
print(f"Correção sugerida:\n{fix}")


Correção sugerida:
public int divide(int a, int b) { 
    if(b == 0) {
        throw new ArithmeticException("Divisão por zero não é permitida");
    }
    return a / b; 
}

Explicação: 

Adicionei uma verificação de condição para garantir que o denominador 'b' não seja zero antes de prosseguir com a divisão. Se 'b' for zero, o código lançará uma exceção de ArithmeticException com uma mensagem adequada. Isso ajudará a evitar a situação de divisão por zero, que é indefinida matematicamente e causará um erro de tempo de execução em muitas linguagens de programação.


DataFrame

In [ ]:
df["Code Fix"] = df.apply(
    lambda row: generate_fix(row["Issue Type"], row["SourceCode"], row["Comment"]),
    axis=1
)

# Salvar o arquivo corrigido
output_path = "/content/corrected_code_fixed.csv"
df.to_csv(output_path, index=False)

print(f"Arquivo corrigido salvo em: {output_path}")

from google.colab import files
files.download(output_path)


Arquivo corrigido salvo em: /content/corrected_code_fixed.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Teste de chaves caso KEYS expirem

In [ ]:
import openai

# Configure sua Admin Key
openai.api_key = "YOUR KEY"

# Configure o Organization ID
openai.organization = "YOUR ORG"  # Substitua pelo seu ID da organização

# Teste de conexão com o modelo GPT
try:
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Use "gpt-4" se disponível
        messages=[
            {"role": "system", "content": "Teste de conexão com o Organization ID."},
            {"role": "user", "content": "Valide esta Admin Key e o modelo GPT usando o Organization ID."}
        ],
        max_tokens=5  # Limite de tokens para economizar
    )
    print("Conexão bem-sucedida! Resposta da API:")
    print(response)
except Exception as e:
    print("Erro ao tentar conectar à API:")
    print(e)
